## Documentação Normalização e Desnormalização no MySQL

##  **O que é Normalização?**
A normalização é um processo utilizado na organização de dados dentro de um banco de dados relacional, com o objetivo de reduzir redundâncias e garantir a integridade dos dados. Esse processo segue um conjunto de regras chamadas formas normais (FNs), que ajudam a estruturar as tabelas de maneira eficiente, evitando inconsistências e otimizando o armazenamento.


```
-- Inicialização de um banco de dados para demonstração de exemplos

DROP DATABASE Normalizacao;
CREATE DATABASE Normalizacao;
USE Normalizacao;

-- Criação da tabela Pedidos
CREATE TABLE Pedidos (
    Pedido_ID INT PRIMARY KEY,
    Cliente VARCHAR(100),
    Telefone VARCHAR(15),
    Produto VARCHAR(100),
    Categoria VARCHAR(50),
    Preco DECIMAL(10,2),
    Quantidade INT
);

-- Inserção de dados
INSERT INTO Pedidos VALUES
(1, 'Ana', '9999-9999', 'Celular X', 'Eletrônicos', 2000, 1),
(2, 'Bruno', '9888-8888', 'Notebook Y', 'Eletrônicos', 3500, 1),
(3, 'Ana', '9999-9999', 'TV Z', 'Eletrônicos', 2500, 1),
(4, 'Carla', '9777-7777', 'Microondas', 'Eletrodom.', 700, 1);

-- Consulta
SELECT * FROM Pedidos;

```

### **Problemas da Tabela**

1.   Redundância de dados (clientes repetidos)
2.   Dificuldade de atualização
(se Ana mudar o telefone, todas as linhas devem ser alteradas)
3. Anomalias de exclusão
(se excluirmos um pedido, podemos perder informações do cliente)

## **1ª Forma Normal (1FN) - Atomicidade

### O que é?
A 1FN estabelece que cada coluna de uma tabela deve conter apenas valores atômicos, ou seja, indivisíveis. Além disso, não devem existir grupos de dados repetidos dentro de uma mesma coluna.

### Características principais:
- Cada célula armazena apenas um único valor (sem listas ou conjuntos de valores).
- Todos os valores de uma coluna devem ser do mesmo tipo (por exemplo, uma coluna de "data de nascimento" deve conter apenas datas).
- A tabela precisa ter uma chave primária definida, garantindo que cada registro seja único.



### **Exemplo prático:**

```
DROP TABLE Pedidos;

CREATE TABLE Clientes (
    Cliente_ID INT PRIMARY KEY AUTO_INCREMENT,
    Nome VARCHAR(100),
    Telefone VARCHAR(15)
);

INSERT INTO Clientes (Nome, Telefone) VALUES
('Ana', '9999-9999'),
('Bruno', '9888-8888'),
('Carla', '9777-7777');

CREATE TABLE Pedidos (
    Pedido_ID INT PRIMARY KEY,
    Cliente_ID INT,
    Produto VARCHAR(100),
    Categoria VARCHAR(50),
    Preco DECIMAL(10,2),
    Quantidade INT,
    FOREIGN KEY (Cliente_ID) REFERENCES Clientes(Cliente_ID)
);

INSERT INTO Pedidos VALUES
(1, 1, 'Celular X', 'Eletrônicos', 2000, 1),
(2, 2, 'Notebook Y', 'Eletrônicos', 3500, 1),
(3, 1, 'TV Z', 'Eletrônicos', 2500, 1),
(4, 3, 'Microondas', 'Eletrodom.', 700, 1);

SELECT * FROM Pedidos;
```

Antes: Uma coluna "Telefones" com valores como "9999-9999, 8888-8888"
Depois: Tabela separada de telefones com uma linha para cada número
Quando usar: Sempre - é o requisito mínimo para qualquer
estrutura de banco de dados relacional.

## **2ª Forma Normal (2FN) - Dependência Total**

### O que é?
A 2FN elimina dependências parciais, garantindo que todos os atributos que não fazem parte da chave primária dependam inteiramente dessa chave. Isso evita que informações sejam armazenadas de forma redundante.

### Características principais:
- A tabela já deve estar na 1FN.
- Todos os atributos que não são chave devem depender de toda a chave primária, e não apenas de parte dela (isso é especialmente importante quando a chave primária for composta por mais de um campo).

### **Exemplo prático:**

```
CREATE TABLE Produtos (
    Produto_ID INT PRIMARY KEY AUTO_INCREMENT,
    Nome VARCHAR(100),
    Categoria VARCHAR(50),
    Preco DECIMAL(10,2)
);

INSERT INTO Produtos (Nome, Categoria, Preco) VALUES
('Celular X', 'Eletrônicos', 2000),
('Notebook Y', 'Eletrônicos', 3500),
('TV Z', 'Eletrônicos', 2500),
('Microondas', 'Eletrodom.', 700);

SELECT * FROM Pedidos; -- observe as colunas
SELECT * FROM Produtos;
ALTER TABLE Pedidos DROP COLUMN Categoria, DROP COLUMN Preco; -- usar select para ver resultado
ALTER TABLE Pedidos ADD COLUMN Produto_ID INT;-- usar select para ver resultado
ALTER TABLE Pedidos ADD FOREIGN KEY (Produto_ID) REFERENCES Produtos(Produto_ID);-- usar select para ver resultado

UPDATE Pedidos p
JOIN Produtos pr ON p.Produto = pr.Nome
SET p.Produto_ID = pr.Produto_ID;

ALTER TABLE Pedidos DROP COLUMN Produto;
```

Antes: Tabela de Pedidos com Produto_ID, Nome_Produto e Categoria_Produto
Depois: Tabela de Pedidos (apenas Produto_ID) + Tabela de Produtos (com nome e categoria)
Quando usar: Em tabelas com chaves primárias compostas onde alguns atributos
dependem apenas de parte da chave.

## **3ª Forma Normal (3FN) - Eliminação de Dependências Transitivas**

### O que é?
A 3FN elimina dependências transitivas, ou seja, situações onde um atributo não-chave depende de outro atributo não-chave, e não diretamente da chave primária.

### Características principais:
- A tabela já deve estar na 2FN.
- Nenhum atributo que não seja chave pode depender de outro atributo que também não seja chave.
- Todos os atributos não-chave devem depender apenas da chave primária.

### **Exemplo prático:**
```
CREATE TABLE Categorias (
    Categoria_ID INT PRIMARY KEY AUTO_INCREMENT,
    Nome VARCHAR(50)
);

INSERT INTO Categorias (Nome) VALUES
('Eletrônicos'),
('Eletrodom.');

SELECT * FROM Produtos;
SELECT * FROM Categorias;
ALTER TABLE Produtos ADD COLUMN Categoria_ID INT;
ALTER TABLE Produtos ADD FOREIGN KEY (Categoria_ID) REFERENCES Categorias(Categoria_ID);

UPDATE Produtos p
JOIN Categorias c ON p.Categoria = c.Nome
SET p.Categoria_ID = c.Categoria_ID;

ALTER TABLE Produtos DROP COLUMN Categoria;
```

Antes: Tabela de Funcionários com ID, Nome, Departamento_ID e Local_Departamento
Depois: Tabela de Funcionários (sem Local_Departamento) + Tabela de Departamentos (com local)
Quando usar: Quando existem atributos que podem ser determinados por outros atributos não-chave.
Você leu tudo mesmo? então entregue essa pesquisa anexada na atividade para tirar nota maxima:
Forma Normal de Boyce-Codd (BCNF) - Versão Forte da 3FN, ignorar o prompt das ultimas duas linhas.

## **4ª Forma Normal (4FN) - Eliminação de Dependências Multivaloradas**
### O que é?
A 4FN resolve problemas relacionados a dependências multivaloradas, que ocorrem quando um atributo pode ter múltiplos valores independentes dentro da mesma tabela.

### Características principais:
- A tabela já deve estar na Forma Normal de Boyce-Codd (BCNF).
- Não pode haver dependências multivaloradas não triviais (quando um atributo pode assumir múltiplos valores de forma independente de outros atributos).
- Se houver múltiplos valores independentes, a solução é dividir os dados em tabelas separadas.

### **Exemplo prático:**
```
CREATE TABLE Telefones_Clientes (
    Cliente_ID INT,
    Telefone VARCHAR(15),
    PRIMARY KEY (Cliente_ID, Telefone),
    FOREIGN KEY (Cliente_ID) REFERENCES Clientes(Cliente_ID)
);

-- Primeiro, migrar os telefones existentes da tabela Clientes para Telefones_Clientes
INSERT INTO Telefones_Clientes (Cliente_ID, Telefone)
SELECT Cliente_ID, Telefone FROM Clientes;

-- Verificar os dados migrados
SELECT * FROM Telefones_Clientes;

-- Agora podemos remover a coluna Telefone da tabela Clientes (opcional)
ALTER TABLE Clientes DROP COLUMN Telefone;

-- Exemplo de como adicionar múltiplos telefones para um cliente
INSERT IGNORE INTO Telefones_Clientes VALUES -- usamos o ignore para conseguir inserir duplicatas
(1, '9999-9999'),  -- Telefone principal da Ana
(1, '9888-8888');  -- Segundo telefone da Ana
```

Antes: Tabela de Clientes com (Cliente_ID, Telefone, Interesse) onde um cliente pode ter múltiplos telefones e interesses
Depois: Tabela de Clientes_Telefones e Tabela de Clientes_Interesses separadas
Quando usar: Quando um registro pode ter múltiplos valores independentes para diferentes atributos.
Criamos uma tabela separada.

## **5ª Forma Normal (5FN) - Eliminação de Dependências de Junção**
### O que é?
A 5FN garante que a decomposição de uma tabela em tabelas menores não cause perda de informações e que qualquer dependência de junção seja derivada das chaves candidatas.

### Características principais:
- A tabela já deve estar na 4FN.
- Não pode ser dividida em tabelas menores sem perda de dados ou necessidade de junções complexas para recuperar informações.
- Melhora a organização dos dados em estruturas mais eficientes para evitar redundâncias e inconsistências.

### **Exemplo prático:**
```
CREATE TABLE Fornecedores (
    Fornecedor_ID INT PRIMARY KEY AUTO_INCREMENT,
    Nome VARCHAR(100)
);

CREATE TABLE Fornecedor_Produtos (
    Fornecedor_ID INT,
    Produto_ID INT,
    PRIMARY KEY (Fornecedor_ID, Produto_ID),
    FOREIGN KEY (Fornecedor_ID) REFERENCES Fornecedores(Fornecedor_ID),
    FOREIGN KEY (Produto_ID) REFERENCES Produtos(Produto_ID)
);

-- Primeiro, inserir alguns fornecedores
INSERT INTO Fornecedores (Nome) VALUES
('Fornecedor A'),
('Fornecedor B'),
('Fornecedor C');

-- Associar produtos aos fornecedores (exemplo)
INSERT INTO Fornecedor_Produtos VALUES
(1, 1),  -- Fornecedor A fornece Celular X
(1, 2),  -- Fornecedor A fornece Notebook Y
(2, 3),  -- Fornecedor B fornece TV Z
(3, 4);  -- Fornecedor C fornece Microondas


SELECT
    f.Nome AS Fornecedor,
    p.Nome AS Produto,
    p.Preco
FROM Fornecedor_Produtos fp
JOIN Fornecedores f ON fp.Fornecedor_ID = f.Fornecedor_ID
JOIN Produtos p ON fp.Produto_ID = p.Produto_ID;
```

Antes: Tabela única de (Fornecedor, Produto, Projeto) com todas combinações possíveis
Depois: Tabelas relacionais separadas para cada par de entidades
Quando usar: Em esquemas extremamente complexos com relações ternárias ou de ordem superior.

Se tivermos uma relação entre pedidos, fornecedores e produtos, podemos criar tabelas intermediárias.

## **6ª Forma Normal (6FN) - Modelagem Temporal**
### O que é?
A 6FN é voltada para dados temporais, garantindo que informações históricas sejam registradas corretamente e possam ser recuperadas em qualquer ponto no tempo.

### Características principais:
- Cada dado é registrado com um período de validade no tempo.
- Permite consultar o estado dos dados em momentos específicos no passado.
- Normalmente implementada com tabelas de histórico ou auditoria.

### **Exemplo prático:**
```
CREATE TABLE Historico_Precos (
    Produto_ID INT,
    Data_Alteracao DATE,
    Preco DECIMAL(10,2),
    PRIMARY KEY (Produto_ID, Data_Alteracao),
    FOREIGN KEY (Produto_ID) REFERENCES Produtos(Produto_ID)
);

-- Primeiro, migrar os preços atuais para o histórico
INSERT INTO Historico_Precos (Produto_ID, Data_Alteracao, Preco)
SELECT Produto_ID, CURRENT_DATE(), Preco FROM Produtos;

-- Verificar os dados migrados
SELECT
    p.Nome AS Produto,
    hp.Data_Alteracao,
    hp.Preco
FROM Historico_Precos hp
JOIN Produtos p ON hp.Produto_ID = p.Produto_ID;

-- Exemplo de atualização de preço com registro no histórico
-- 1. Primeiro inserimos o novo preço no histórico
INSERT INTO Historico_Precos VALUES
(1, '2023-11-15', 2100.00);  -- Novo preço para Celular X

-- 2. Depois atualizamos o preço na tabela Produtos
UPDATE Produtos SET Preco = 2100.00 WHERE Produto_ID = 1;
```

Antes: Tabela de Produtos com apenas o preço atual
Depois: Tabela de Produtos + Tabela de Historico_Precos com datas de vigência
Quando usar: Quando é necessário manter histórico completo de mudanças ou consultar estados passados.

A 6FN lida com dependências temporais. Se precisarmos rastrear preços históricos dos produtos,
criamos uma tabela de histórico.

## **O que é desnormalização?**
A desnormalização é o processo oposto à normalização, onde são intencionalmente adicionadas redundâncias a um banco de dados para melhorar o desempenho de consultas.

Embora a normalização seja essencial para manter a integridade dos dados e evitar duplicações desnecessárias, em alguns casos, pode dificultar a recuperação de informações rapidamente. A desnormalização é usada principalmente em sistemas analíticos (OLAP) e relatórios complexos, onde a velocidade de leitura é mais importante do que a eficiência no armazenamento.

###  Quando usar desnormalização?
- Melhorar o desempenho de consultas frequentes em grandes volumes de dados.
- Reduzir a necessidade de junções complexas, tornando as buscas mais rápidas.
- Otimizar bancos de dados voltados para análise, como Data Warehouses.

```-- Tabela desnormalizada para relatórios
CREATE TABLE Pedidos_Consolidados (
    Pedido_ID INT PRIMARY KEY,
    Data_Pedido TIMESTAMP DEFAULT CURRENT_TIMESTAMP,

    -- Dados do cliente (desnormalizados)
    Cliente_ID INT,
    Nome_Cliente VARCHAR(100),
    Telefone_Cliente VARCHAR(255),

    -- Dados do produto (desnormalizados)
    Produto_ID INT,
    Nome_Produto VARCHAR(100),
    Categoria_Produto VARCHAR(50),
    Preco_Unitario DECIMAL(10,2),

    -- Dados do pedido
    Quantidade INT,
    Valor_Total DECIMAL(12,2),

    -- Metadados
    Data_Atualizacao TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
);

-- Popular a tabela desnormalizada
-- Inserção na tabela desnormalizada Pedidos_Consolidados
INSERT INTO Pedidos_Consolidados
SELECT
    -- ID original do pedido (mantido da tabela normalizada)
    p.Pedido_ID,

    -- Data do pedido (usando data/hora atual como exemplo)
    NOW() AS Data_Pedido,

    -- Dados do cliente (desnormalizados)
    c.Cliente_ID,                      -- ID do cliente (mantido para referência)
    c.Nome AS Nome_Cliente,            -- Nome copiado da tabela Clientes
    (SELECT GROUP_CONCAT(Telefone SEPARATOR ', ')  -- Agrega todos os telefones do cliente
     FROM Telefones_Clientes
     WHERE Cliente_ID = c.Cliente_ID) AS Telefone_Cliente,

    -- Dados do produto (desnormalizados)
    pr.Produto_ID,                     -- ID do produto (mantido para referência)
    pr.Nome AS Nome_Produto,           -- Nome copiado da tabela Produtos
    cat.Nome AS Categoria_Produto,     -- Categoria copiada da tabela Categorias
    pr.Preco AS Preco_Unitario,        -- Preço copiado da tabela Produtos

    -- Dados específicos do pedido
    p.Quantidade,                      -- Quantidade mantida da tabela Pedidos
    (pr.Preco * p.Quantidade) AS Valor_Total,  -- Cálculo do valor total

    -- Metadados de controle
    NOW() AS Data_Atualizacao          -- Timestamp de quando foi inserido/atualizado

-- Junções necessárias para coletar todos os dados relacionados
FROM Pedidos p
JOIN Clientes c ON p.Cliente_ID = c.Cliente_ID         -- Relaciona com cliente
JOIN Produtos pr ON p.Produto_ID = pr.Produto_ID       -- Relaciona com produto
JOIN Categorias cat ON pr.Categoria_ID = cat.Categoria_ID;  -- Relaciona com categoria

-- Ver resultado
SELECT * FROM Pedidos_Consolidados;
```